## Example: BAO and $\sigma_8(z)$ forecasts


$\verb|FishLSS|$ requires $\verb|velocileptors|$ to run, which can be installed with: `python3 -m pip install -v git+https://github.com/sfschen/velocileptors`.

In [1]:
# import revelant packages
from headers import *
from twoPoint import *
from twoPointNoise import *

## (1) Setting up a $\verb|FishLSS|$ forecast

### (1a) The cosmology

A $\verb|FishLSS|$ forecast requires two main ingredients: a fiducial cosmology and an experiment. For the input cosmology, we use a $\verb|CLASS|$ object:

In [ ]:
params = {'output': 'tCl lCl mPk',
          'l_max_scalars': 1000,
          'lensing': 'yes',
          'P_k_max_h/Mpc': 2.,
          'non linear':'halofit', 
          'z_pk': '0.0,6',
          'A_s': 2.10732e-9,
          'n_s': 0.96824,
          'alpha_s': 0.,
          'h': 0.6770,
          'N_ur': 1.0196,
          'N_ncdm': 2,
          'm_ncdm': '0.01,0.05',
          'tau_reio': 0.0568,
          'omega_b': 0.02247,
          'omega_cdm': 0.11923,
          'Omega_k': 0.}

cosmo = Class() 
cosmo.set(params) 
cosmo.compute() 

### (1b) The experiment

Now we specify the experiment, which is an instance of $\verb|experiment.py|$. At a minimum, we need to specify the redshift range of the survey ($z_\text{min}$ and $z_\text{max}$), the redshift binning, the sky coverage $f_\text{sky}$, the linear bias $b(z)$, and the number density $\bar{n}(z)$. The redshift binning can be specified in two ways: you can either input a `zedges` (numpy array) to specify the edges of the bins, or `nbins` (integer), in which case the redshift bins are assumed to be linearly spaced in $z$. Here we set `nbins = 3` for simplicity, so that we have three redshift bins with $\Delta z=1$.

In [3]:
# minimal set of inputs to an experiment object
zmin = 2.
zmax = 5.
nbins = 3
fsky = 0.5
b = lambda z: (1+z) 
n = lambda z: 3e-4 * np.exp(-(z-3.5)**2)

We can now create the experiment:

In [4]:
exp = experiment(zmin=zmin, zmax=zmax, nbins=nbins, fsky=fsky, b=b, n=n)

In addition to the above, one can optionally specify the following in an `experiment` object.

- `b2` (function of $z$): quadratic bias $b_2(z)$ of the tracer, default $b_2 = 8(b-1)/21$
- `sigv` (float): the comoving velocity dispersion for FoG contributions [km/s], default is 100 km/s
- `sigma_z` (float): redshift error $\sigma_z/(1+z)$, assumed to be independent of redshift, default is 0

### (1c) The forecast object

With a cosmology and an experiment in hand, we can now create a forecast. Running the line below will create an `output` directory, as well as a subdirectory for the experiment of interest: `output/example` in this case. After creating these directories, $\verb|FishLSS|$ will calculate the fiducial power spectra ($P_{gg}(\boldsymbol{k},z)$ and $P_\text{recon}(\boldsymbol{k},z)$) at the center of each redshift bin, and store them in `output/example/derivatives/` and `output/example/derivatives_recon` respectively (assuming that the files don't already exist). $\verb|FishLSS|$ will also calculate $C^{\kappa\kappa}_\ell$, $C^{\kappa g_i}_\ell$ and $C^{g_i g_i}_\ell$ for each redshift bin, and save them in `output/example/derivatives_Cl`. 

Calculating the fiducial power spectra takes a little while ($\sim 10$ minutes for $3$ redshift bins), so go grab some coffee...

In [5]:
forecast = fisherForecast(experiment=exp,cosmo=cosmo,name='example')

Redshift-space power spectra are computed on a (flattened) $k-\mu$ grid. That is, $P_{gg}(\boldsymbol{k},z)$ is stored as an array of length `forecast.Nk * forecast.Nmu`, with the corresponding values of $k$ and $\mu$ stored in `forecast.k` and `forecast.mu`. 

## (2) BAO forecast

As described in $\S3.6$ of [2106.09713](https://arxiv.org/pdf/2106.09713.pdf), we hold the shape of the fiducial power spectrum fixed in our BAO forecasts. We then find the errors on the two A-P parameters ($\alpha_\perp$, $\alpha_\parallel$) after marginalizing over the linear bias $b$ and 15 broad-band polynomials $\sum_{n=0}^4\sum_{m=0}^2 c_{nm}k^n\mu^{2m}$. We finally intepret the errors on the A-P parameters as the relative errors of $D_A(z)/r_d$ and $H(z)r_d$, where $r_d$ is sound horizon at the drag epoch.

Marginalizing over the polynomial coefficients is trivial to do analytically, so we only need to numerically compute derivatives with respect to $\alpha_\perp,\alpha_\parallel$ and $b$. Calculating these derivatives will take a while.

In [6]:
basis = np.array(['alpha_perp','alpha_parallel','b'])

# set recon = True, so that we perform BAO reconstruction when computing the power spectrum
forecast.recon = True

# set the "marginalized parameters", aka the derivatives, to be [alpha's, linear b]
forecast.marg_params = basis

In [ ]:
# computes the derivatives in each redshift bin (evaluated at the midpoint of each bin)
forecast.compute_derivatives()

The derivatives have been automatically stored in `output/example/derivatives_recon`. To load these derivatives, simply run:

In [7]:
derivs = forecast.load_derivatives(basis)

Now let's compute the Fisher matrices in each redshift bin using `get_fisher`, which takes the arguments:

- `basis` (np array): basis of the Fisher matrix 
- `globe` (int): let's not worry about this for now, it's value isn't important for computing the Fisher matrix for a single redshift bin
- `derivatives` (np array): load the derivatives from memory, if not specificied $\verb|FishLSS|$ will recalculate them (takes a lot of time!)
- `zbins` (np array): an array of ints specifying which redshift bins to include in the Fisher matrix. In this case we're computing a Fisher matrix for each redshift bin, so we set `zbins = np.array([i])` to get the Fisher matrix for the i'th bin.

In addition the the above you can also specify `kmax` or `kmax_knl` (ratio of $k_\text{max}$ to the non-linear scale at the center of each redshift bin). By default we set `kmax_knl=1` and `kmax=-10`. If `kmax` is set to be a positive number, then the code ignores `kmax_knl` and uses `kmax` instead.

In [8]:
# get the fisher matrices in each of the 3 redshift bins
F = lambda i: forecast.gen_fisher(basis, 100, derivatives=derivs, zbins=np.array([i]))
Fs = [F(i) for i in range(nbins)]
Fs = np.array(Fs)

Since we set `recon = True` when computing the Fisher matrices, $\verb|FishLSS|$ automatically knows to marginalize over the 15 polynomials, so each Fisher matrix will be an $18\times18$ matrix with basis $\{\alpha_\perp,\alpha_\parallel,b,c_{00},\cdots\}$

In [9]:
print(Fs.shape)

(3, 18, 18)


Now lets invert and compute the errors on the A-P parameters.

In [10]:
Finvs = [np.linalg.inv(Fs[i]) for i in range(nbins)]
saperp = [np.sqrt(Finvs[i][0,0]) for i in range(nbins)]
saparr = [np.sqrt(Finvs[i][1,1]) for i in range(nbins)]

From which we get the relative error on $D_A(z)/r_d$ and $H(z)r_d$ in each redshift bin:

In [11]:
print('Relative error on DA/rd:',saperp)
print('Relative error on H*rd:',saparr)

Relative error on DA/rd: [0.003181433656693976, 0.001842152405859197, 0.003276545170228536]
Relative error on H*rd: [0.004565731326440649, 0.0028150515717583935, 0.004831631731011703]


We're done with BAO forecasting, so let's set `recon = False`

In [12]:
forecast.recon = False

## (3) $\sigma_8(z)$ forecast

### (3a) From full-shape data only

#### Derived from $\Lambda$CDM

Here we'll compute the relative error on $\sigma_8(z)$ in each of the three redshift bins, following the methods described in $\S4.3$ of [2106.09713](https://arxiv.org/pdf/2106.09713.pdf). First we have to compute the relevant derivatives of the full shape power spectrum. This takes an hour-ish.

In [13]:
basis = np.array(['h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio',
                  'N','alpha0','b','b2','bs','N2','N4','alpha2','alpha4'])

forecast.marg_params = basis

In [ ]:
forecast.compute_derivatives()

Let's load the derivatives

In [14]:
derivatives = forecast.load_derivatives(basis)

and compute the Fisher matrices in each bin, which has a 15-dimensional basis $$\{h,\ln(A_s),n_s,\omega_c,\omega_b,\tau,N_0(z_i),\alpha_0(z_i),b(z_i),b_2(z_i),b_s(z_i),N_2(z_i),N_4(z_i),\alpha_2(z_i),\alpha_4(z_i)\}$$
The order that I chose for the nuisance terms in this basis might seem odd to you, but this will become more clear when combining full shape information with CMB lensing.

In [15]:
F = lambda i: forecast.gen_fisher(basis,100,kmax_knl=1.,derivatives=derivatives, zbins = np.array([i]))
Fs = np.array([F(i) for i in range(nbins)])

Finally, we are going to add a Gaussian prior on $\omega_b$ from BBN. We do this using the `combine_fishers` function, which takes the following inputs:

- a list of fisher matrices (they don't all have to be the same dimension)
- `globe` (int): Let $\{a_1,\cdots,a_n,b_1,\cdots,b_m\}$ be the basis of the first Fisher matrix and $\{a_1,\cdots,a_n,c_1,\cdots,c_l\}$ be the basis of the second Fisher matrix, where I'm assuming that the $b_i$'s and $c_i$'s are distinct parameters (such as two different sets of nuisance parameters), then `globe = n`. That is, `globe` counts the number of "global" parameters that are common to the Fisher matrices. In this case we are combining a $6\times 6$ Fisher with a $15\times 15$ Fisher, with the first six parameters being the same $\Lambda$CDM parameters for both Fishers. So for this case we set `globe = 6`. <span style="color:red"> This function assumes that the Fishers have the same basis up to the globe'th entry. </span>

In [16]:
BBN_prior = np.array([0,0,0,0,(5e-4)**(-2),0])
BBN_prior = np.diag(BBN_prior)

for i in range(nbins): Fs[i] = forecast.combine_fishers([Fs[i],BBN_prior],6)

We want to rotate from the basis $\{h,\ln(A_s),n_s,\omega_\text{cdm},\omega_b,\tau,\cdots\}$ to $\{\sigma_8(z_i),\ln(A_s),n_s,\omega_\text{cdm},\omega_b,\tau,\cdots\}$, where $z_i$ is the central redshift in the i'th redshift bin. To do this we need to compute the rotation matrix, which we do using the `rotation_matrix` helper function below.

In [17]:
oldbasis = np.array(['h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio'])

def s8(z):
   s8 = cosmo.sigma(8/params['h'],z)
   return s8 

def ds8dp(param,z):
   flag = False
   if param == 'h' or param == 'n_s' or param == 'omega_cdm' or param == 'omega_b' or param == 'tau_reio' or param =='log(A_s)':
      if param == 'log(A_s)':
         flag = True
         param = 'A_s'
      fid_val = params[param]
      fid_fs8 = s8(z)
      cosmo.set({param: fid_val*1.01})
      cosmo.compute()
      new_fs8 = s8(z)
      cosmo.set({param: fid_val})
      cosmo.compute()
      if flag: return params[param]*(new_fs8-fid_fs8)/fid_val/0.01
      return (new_fs8-fid_fs8)/fid_val/0.01
   else: return 0

def rotation_matrix(z,n):
   result = np.zeros((n,n))
   for i in range(n):
      for j in range(n):
         if i != 0 and i == j: result[i,j] = 1
         if i == 0 and j<len(oldbasis): result[i,j] = ds8dp(oldbasis[j],z)
   return result

Here are the rotation matrices for each of the three redshift bins:

In [18]:
Rs_example = np.array([rotation_matrix(z,n=15) for z in forecast.experiment.zcenters])

Here's a helper function to do the rotation, and to get the relative constraints on $\sigma_8(z)$

In [19]:
def get_rel_s8_constraint(Fs,Rs):
   con = np.zeros(len(Rs)) # len(Rs) = number of redshift bins
   for i in range(len(Rs)):
      F = Fs[i] ; Rinv = np.linalg.inv(Rs[i])
      Fprime = np.dot(np.dot(Rinv.T,F),Rinv) # rotate to new basis
      Cprime = np.linalg.inv(Fprime)
      zi = forecast.experiment.zcenters[i]
      con[i] = np.sqrt(Cprime[0,0])/s8(zi) # get constraint on sigma8(z)
   return con

In [20]:
s8_constraints = get_rel_s8_constraint(Fs,Rs_example)
print('Relative constraints on sigma8(z) (derived from LCDM):',s8_constraints)

Relative constraints on sigma8(z) (derived from LCDM): [0.01912116 0.0148741  0.02459258]


#### Fixed shape

Now we are going to get the errors on $\sigma_8(z)$ assuming that the shape of the power spectrum is fixed. In this case $\sigma_8^2\propto A_s$ is entirely determined by the primordial amplitude $A_s$. At the level of derivatives this implies $\partial_{\ln \sigma_8(z)} = 2\partial_{\ln A_s}$, so the relative error on $\sigma_8(z)$ is half the relative error on $A_s$. 

In [21]:
basis_fixed = np.array(['log(A_s)','N','alpha0','b','b2','bs','N2','N4','alpha2','alpha4'])
derivatives_fixed = forecast.load_derivatives(basis_fixed)

F_fix = lambda i: forecast.gen_fisher(basis_fixed,100,kmax_knl=1.,
                derivatives=derivatives_fixed, zbins = np.array([i]))

Fs_fixed = np.array([F_fix(i) for i in range(nbins)])
Finvs_fixed = np.array([np.linalg.inv(Fs_fixed[i]) for i in range(nbins)])

In [22]:
def get_s8_constraint(i): return np.sqrt(Finvs_fixed[i][0,0])/2 # divide by two to convert A -> s8
s8_constraints_fixed = np.array([get_s8_constraint(i) for i in range(nbins)])

print('Relative constraints on sigma8(z) (fixed shape):',s8_constraints_fixed)

Relative constraints on sigma8(z) (fixed shape): [0.01072835 0.00743769 0.01343389]


### (3b) Computing CMB lensing $\times$ galaxies Fisher matrices

We first need to compute the relevant derivatives of $(C^{\kappa\kappa}_\ell, C^{\kappa g_i}_\ell, C^{g_ig_i}_\ell)$. Again, this will take a while.

In [23]:
basis_lensing = np.array(['h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio',\
                          'N','alpha0','b','b2','bs','alphax'])
forecast.marg_params = basis_lensing

In [ ]:
forecast.compute_Cl_derivatives()

Now we'll compute the CMB lensing $\times$ galaxies Fisher matrices in each bin using `gen_lensing_fisher`, which takes the following inputs:

- `basis_lensing` (np array): the basis of the Fisher matrix
- `globe_lensing` (np array): number of global parameters (parameters which don't depend on the redshift bin), in this case 6 ($\Lambda$CDM)
- `ell_min`, `ell_max` (ints): multipoles to include in the Fisher matrix
- `bins` (np array): array of ints to specify which redshift bins to include in the Fisher matrix, default is to include all the redshift bins
- `kk` (boolean): Set False to remove $C^{\kappa\kappa}_\ell$ from the data vector, default is True
- `CMB` (string): Choose from 'Planck', 'SO' or 'S4' to set lensing noise levels 

In our example, the lensing Fisher matrix will have basis $\{h,\ln(A_s),n_s,\omega_c,\omega_b,\tau,N_0(z_1),\cdots,N_0(z_n),\alpha_0(z_1),\cdots,\alpha_0(z_n),b(z_1),\cdots\}$, where $z_i$ is the central redshift of the i'th bin. In this example $i=0,1,2$. This is true regardless of the `bins` input. However, if we're only including one redshift bin in our data vector, then our data is obviously insensitive to the nuisance parameters in the other bins. Thus when setting `bins = np.array([i])`, we can remove the $N_0(z_j),\alpha_0(z_j),\cdots$ ($j\neq i$) columns/rows from the Fisher matrix (which are all zero), so that our basis becomes: 
$$\{h,\ln(A_s),n_s,\omega_c,\omega_b,\tau,N_0(z_i),\alpha_0(z_i),b(z_i),b_2(z_i),b_s(z_i),\alpha_x(z_i)\}$$
Below is a helper function `get_lensing_fishers` which deletes these unnessecary rows. 

In [24]:
def get_lensing_fishers(cast):
   globe_lensing = 6
   # xs is a set of indicies telling me the relevant terms for the i'th redshift bin
   xs = [list(range(6)) # LCDM
         +
         [int(6+0*nbins+i),int(6+1*nbins+i),int(6+2*nbins+i), # relevant nuisance terms (6 of them)
          int(6+3*nbins+i),int(6+4*nbins+i),int(6+5*nbins+i)]
         for i in range(nbins)]
   Lensing = [forecast.gen_lensing_fisher(basis_lensing,globe_lensing,ell_min=30,ell_max=500,
                                          bins=np.array([i]),kk=False,CMB='S4') for i in range(nbins)]
   Short = [Lensing[i][xs[i]][:,xs[i]] for i in range(nbins)] # remove all off the irrelevant columns/rows
   return np.array(Short)

In [25]:
Fs_lensing = get_lensing_fishers(forecast)

### (3c) $\sigma_8(z)$ constraints from full-shape and CMB lensing $\times$ galaxies

#### Derived from $\Lambda$CDM

The bases for our full-shape and CMB lensing $\times$ galaxies Fishers are (for each individual redshift bin):

$$\text{Full-shape}=G + \{N_2(z_i),N_4(z_i),\alpha_2(z_i),\alpha_4(z_i)\}$$
$$\text{CMB lensing}\times\text{galaxies}=G + \{\alpha_x(z_i)\}$$
where $G$ (the global piece) is defined to be
$$G = \{h,\ln(A_s),n_s,\omega_c,\omega_b,\tau,N_0(z_i),\alpha_0(z_i),b(z_i),b_2(z_i),b_s(z_i)\}.$$
To combine the two Fishers we can just use the `combine_fishers` function, with `globe = 11` (the size of $G$). We do this for the "derived from $\Lambda$CDM" procedure in the bit of code below. The combined Fisher matrix has basis
$$G + \{N_2(z_i),N_4(z_i),\alpha_2(z_i),\alpha_4(z_i),\alpha_x(z_i)\}$$

In [26]:
Fs_combined = []
for i in range(nbins): 
    fishers = [Fs[i],Fs_lensing[i]] # full-shape, CMB x galaxies
    globe = 11
    combined = forecast.combine_fishers(fishers,globe)
    Fs_combined.append(combined)
Fs_combined = np.array(Fs_combined)

In [27]:
# we'll be lazy and recompute the rotation matrices with the correct dimension (16)
Rs_example = np.array([rotation_matrix(z,n=16) for z in forecast.experiment.zcenters])

In [28]:
s8_constraints_combined = get_rel_s8_constraint(Fs_combined,Rs_example)
print('Relative constraints on sigma8(z) (derived from LCDM, w CMB lensing):',s8_constraints_combined)

Relative constraints on sigma8(z) (derived from LCDM, w CMB lensing): [0.01383381 0.01084548 0.01756993]


#### Fixed shape

The procedure here is nearly identical apart from the global piece $G$, which now doesn't contain $h,n_s,\omega_c,\omega_b,\tau$ (in particular, we now set `globe = 6`). I'm using `xs` to remove the $h,n_s,\omega_c,\omega_b,\tau$ rows/columns from `Fs_lensing`

In [31]:
Fs_combined_fixed = []
for i in range(nbins): 
    xs = [1,6,7,8,9,10,11]
    fishers = [Fs_fixed[i],Fs_lensing[i][xs][:,xs]] # full-shape, CMBxgalaxies
    globe = 6
    combined = forecast.combine_fishers(fishers,globe)
    Fs_combined_fixed.append(combined)
Finvs_combined_fixed = np.array([np.linalg.inv(Fs_combined_fixed[i]) for i in range(nbins)])

In [32]:
def get_s8_constraint(i): return np.sqrt(Finvs_combined_fixed[i][0,0])/2 # divide by two to convert A -> s8
s8_constraints_combined_fixed = np.array([get_s8_constraint(i) for i in range(nbins)])

print('Relative constraints on sigma8(z) (fixed shape, w lensing):',s8_constraints_combined_fixed)

Relative constraints on sigma8(z) (fixed shape, w lensing): [0.00898109 0.00683269 0.01183671]
